In [1]:
from rdflib import Graph
from pydantic import BaseModel, validator, Field
from typing import Optional
from pyfuseki import FusekiUpdate
import json
from datetime import date, datetime

In [49]:
with open('api/src/config/authorized_values.json') as file:
    reader = file.read()
    vocabulary = json.loads(reader)
    file.close()

In [ ]:
vocabulary['relators']

In [2]:
class GenerationProcess(BaseModel):
    label: str = Field(default="BiblioKeia v.1")
    generationDate: Optional[str]

    @validator('generationDate', pre=True, always=True)
    def set_date_now(cls, v):
        now = datetime.now().strftime('%d-%m-%YT%H:%M:%S')
        
        return now

In [9]:
class AdminMetadata(BaseModel):
    encodingLevel: str = Field(default="menclvl:f")
    assigner: str = Field(default="http://id.loc.gov/vocabulary/organizations/brmninpa")
    creationDate: Optional[str]
    descriptionConventions: str = Field(default="http://id.loc.gov/vocabulary/descriptionConventions/isbd")
    descriptionLanguage: str = Field(default="http://id.loc.gov/vocabulary/languages/por")
    generationProcess: Optional[GenerationProcess]
    status: str = Field(default="mstatus:new")

    _status = vocabulary['mstatus']
    
    _level = vocabulary['menclvl']

    @validator('status')
    def status_accepted(cls, v):
        if v not in cls._status:
            raise ValueError(f"the status code must be one of the following : {', '.join(cls._status)}")
        return v
    
    @validator('encodingLevel')
    def level_accepted(cls, v):
        if v not in cls._level:
            raise ValueError(f"the level code must be one of the following : {', '.join(cls._level)}")
        return v

    @validator('creationDate', pre=True, always=True)
    def set_date_today(cls, v):
        today = date.today().strftime("%d-%m-%Y")
        
        return today

In [18]:
class Classification(BaseModel):
    classificationPortion: str
    itemPortion: str
    edition: int

In [73]:
class Contribution(BaseModel):
    type: list[str]
    agent: str
    role: str

    _type = ["bf:Contribution", "bflc:PrimaryContribution" ]
    _role = vocabulary['relators']

    @validator('type')
    def type_accepted(cls, v):
        for i in v:
            if i not in cls._type:
                raise ValueError(f"the type code must be one of the following : {', '.join(cls._type)}")
        return v
    
    @validator('role')
    def role_accepted(cls, v):
        if v not in cls._role:
            raise ValueError(f"the role is not permited")
        return v

In [123]:
class Work(BaseModel):
    type: list[str]
    adminMetadata: AdminMetadata
    classification: Classification
    content: str = Field(default="contentTypes:txt")
    contribution: list[Contribution]
    expressionOf: Optional[str]

    _content = vocabulary['contentTypes']

    @validator('content')
    def content_accepted(cls, v):
        if v not in cls._content:
            raise ValueError(f"the content code must be one of the following : {', '.join(cls._content)}")
        return v

In [107]:
with open('work.json') as file:
    reader = file.read()
    work = json.loads(reader)
    file.close()
work

{'type': ['bf:Monograph', 'bf:Text', 'bf:Work'],
 'adminMetadata': {'encodingLevel': 'menclvl:1',
  'generationProcess': {'label': 'BiblioKeia'}},
 'classification': {'classificationPortion': '869.3',
  'itemPortion': 'M513',
  'edition': 23},
 'content': 'contentTypes:txt',
 'contribution': [{'type': ['bf:Contribution'],
   'agent': 'http://id.loc.gov/rwo/agents/no2015001378',
   'role': 'relators:trl'},
  {'type': ['bf:Contribution', 'bflc:PrimaryContribution'],
   'agent': 'http://id.loc.gov/rwo/agents/n80002329',
   'role': 'relators:aut'}]}

In [125]:
request = Work(**work)
request.dict()

{'type': ['bf:Monograph', 'bf:Text', 'bf:Work'],
 'adminMetadata': {'encodingLevel': 'menclvl:1',
  'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '17-04-2023',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': {'label': 'BiblioKeia',
   'generationDate': '17-04-2023T16:21:33'},
  'status': 'mstatus:new'},
 'classification': {'classificationPortion': '869.3',
  'itemPortion': 'M513',
  'edition': 23},
 'content': 'contentTypes:txt',
 'contribution': [{'type': ['bf:Contribution'],
   'agent': 'http://id.loc.gov/rwo/agents/no2015001378',
   'role': 'relators:trl'},
  {'type': ['bf:Contribution', 'bflc:PrimaryContribution'],
   'agent': 'http://id.loc.gov/rwo/agents/n80002329',
   'role': 'relators:aut'}],
 'expressionOf': None}

In [109]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    
    """

In [118]:
def MakeContribution(contributions):
        listContributions = list()
        for i in contributions:
                c = f"""[ a { ", ".join(i.type) } ;
                bf:agent <{i.agent}> ;
                bf:role {i.role} ]"""
                listContributions.append(c)
        contribution = ", ".join(listContributions)

        return contribution


contribution = MakeContribution(request.contribution)
print(contribution)

[ a bf:Contribution ;
                bf:agent <http://id.loc.gov/rwo/agents/no2015001378> ;
                bf:role relators:trl ], [ a bf:Contribution, bflc:PrimaryContribution ;
                bf:agent <http://id.loc.gov/rwo/agents/n80002329> ;
                bf:role relators:aut ]


In [128]:
id = "bk-2"
graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH bkw:{id}
        {{ 
           bkw:{id} a { ", ".join(request.type) }  ;
           bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel {request.adminMetadata.encodingLevel} ;
            bf:assigner <{request.adminMetadata.assigner}> ;    
            bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;    
            bf:descriptionConventions <{request.adminMetadata.descriptionConventions}> ;
            bf:descriptionLanguage <{request.adminMetadata.descriptionLanguage}> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess.label}" ;
                    bf:generationDate "{request.adminMetadata.generationProcess.generationDate}"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <{request.adminMetadata.assigner}> ;
                    rdf:value "{id}" ] ;
            bf:status {request.adminMetadata.status} ] ;
        bf:classification [ a bf:ClassificationDdc ;
                bf:assigner <{request.adminMetadata.assigner}> ;
                bf:classificationPortion "{request.classification.classificationPortion}" ;
                bf:itemPortion "{request.classification.itemPortion}" ;
                bf:source [ a bf:Source ;
                    bf:code "{request.classification.edition}" ] ] ;
        bf:content {request.content} ;
        bf:contribution  { MakeContribution(request.contribution) }
        { f'bf:expressionOf <{request.expressionOf}> ;' if request.expressionOf else ''}
        }} }}"""

print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    
        
    INSERT DATA {
        GRAPH bkw:bk-2
        { 
           bkw:bk-2 a bf:Monograph, bf:Text, bf:Work  ;
           bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:encodingLevel menclvl:1 ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;    
            bf:creationDate "17-04-2023"^^xsd:date ;    
 

 ns1:assigner <http://id.loc.gov/vocabulary/organizations/dlc> ;
            ns1:changeDate "2023-02-06T08:03:19"^^xsd:dateTime ;
            ns1:creationDate "2019-11-26"^^xsd:date ;
            ns1:descriptionAuthentication <http://id.loc.gov/vocabulary/marcauthen/pcc> ;
            ns1:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/isbd>,
                <http://id.loc.gov/vocabulary/descriptionConventions/rda> ;
            ns1:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            ns1:descriptionModifier <http://id.loc.gov/vocabulary/organizations/dlc> ;
            ns1:generationProcess [ a ns1:GenerationProcess ;
                    rdfs:label "DLC marc2bibframe2 v2.2.1" ;
                    ns1:generationDate "2023-02-07T01:58:10.983465-05:00"^^xsd:dateTime ] ;
            ns1:identifiedBy [ a ns1:Local ;
                    ns1:assigner <http://id.loc.gov/vocabulary/organizations/dlc> ;
                    rdf:value "21313495" ] ;
            ns1:status <http://id.loc.gov/vocabulary/mstatus/c> ;

In [120]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

response = fuseki_update.run_sparql(graph)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exemple Work Loc

In [121]:
g = Graph()
g.parse('http://id.loc.gov/resources/hubs/83ffc803-c205-176c-8646-13e1f4d8587c.nt', format='nt')

<Graph identifier=N04307ee2ed4f49a1a1d92cfd6e93ec01 (<class 'rdflib.graph.Graph'>)>

In [122]:
g.serialize('hub.ttl', format='ttl')

<Graph identifier=N04307ee2ed4f49a1a1d92cfd6e93ec01 (<class 'rdflib.graph.Graph'>)>

POST WORK